# Nonlinearity in Finance

> To-do list:

* Examining the definition of nonlinearity
* Discussing the volatility smile in implied volatility modeling
* Discussing Markov switching models, threshold models, and smooth transition models as nonlinear models
* An overview of root-finding to find the optimal point of nonlinear models
* Examining the incremental search algorithm, bisection algorithm, Newton's algorithm, and secant method in root-finding
* Combining root-finding methods with Brent's method
* SciPy's implementation of root-finding methods as scalar functions
* SciPy's general nonlinear solvers in root-finding


A nonlinear relationship is defined as follows:
$$ f (a+b)≠ f (a)+ f (b) $$

## The Markov regime-switching model

$$ y_t = x_1+ε_t, when\ s_t =1  $$
$$ y_t = x_2+ε_t, when\ s_t =2  $$

thus,

$$ y_t =x_1D_t +x_2(1−D_t)+ε_t $$

where $D_t$ is a binary variable taking value 0 or 1

## The threshold autoregressive model

$$ y_t = a_1+b_1y_{t-d} +ε_t,if\ y_{t-d} ≤c $$
$$ y_t = a_2+b_2y_{t-d} +ε_t,if\ y_{t-d} >c $$

thus,

$$ y_t = (a_1+b_1y_{t-d})D_t+ (a_2+b_2y_{t-d})(1−D_t) +ε_t $$
where $D_t$ is either 1 or 0

## Smooth transition models
logistic function $ G(y_{t−1};γ,c)$:
$$  G(y_{t−1};γ,c)=\frac{1}{1+e^{-γ(y_{t−d}-c)}}  $$

# Intro to root finding

find where $ f(x)=0 $, it can be either maximum or minimum.


## Incremental search

We assume that the values of $ f (a + dx)$, $f (a + 2dx)$, $f (a + 3dx)$... are going in the same direction as indicated by their sign. Once the sign changes, a solution is deemed as found. 

In [3]:
import numpy as np
def incremental_search(f, a, b, dx):
    """
    :param f: The function to solve
    :param a: The left boundary x-axis value
    :param b: The right boundary x-axis value
    :param dx: The incremental value in searching
    :return: The x-axis value of the root,
                number of iterations used
    """
    fa = f(a) # left value
    c = a + dx # point after increment
    fc = f(c) 
    n= 1
    while np.sign(fa) == np.sign(fc): # loop continues if the sings are the same
        if a >= b:
            return a - dx, n
        a= c
        fa = fc
        c = a + dx 
        fc = f(c) 
        n += 1
    if fa == 0:
        return a, n
    elif fc == 0:
        return c, n
    else:
        return (a + c)/2., n

In [4]:

# define a function
y = lambda x: x**3 + 2.0*x**2 - 5
# use incremental search
root, iterations = incremental_search (y, -5., 5., 0.001)

print(root, iterations)

1.2414999999999783 6242


## The bisection method

In [8]:
def bisection(f, a, b, tol=0.1, maxiter=10):
    """
    :param f: The function to solve
    :param a: The x-axis value where f(a)<0
    :param b: The x-axis value where f(b)>0
    :param tol: The precision of the solutio
    """
    c = (a+b)*0.5  # Declare c as the midpoint ab
    n = 1  # Start with 1 iteration
    while n <= maxiter:
        c = (a+b)*0.5
        if f(c) == 0 or abs(a-b)*0.5 < tol:
            # Root is found or is very close
            return c, n
        n += 1
        if f(c) < 0:
            a= c 
        else:
            b= c
    return c, n

In [9]:
y = lambda x: x**3 + 2*x**2 - 5
root, iterations = bisection(y, -5, 5, 0.00001, 100)
print(root, iterations)

1.241903305053711 20


## Newton's method

$$ x_1 = x- \frac{f(x)}{f'(x)} $$

In [10]:
def newton(f, df, x, tol=0.001, maxiter=100):
    """
    :param f: The function to solve
    :param df: The derivative function of f
    :param x: Initial guess value of x
    :param tol: The precision of the solution
    :param maxiter: Maximum number of iterations
    :return: The x-axis value of the root, number of iterations used
    """ 
    n= 1
    while n <= maxiter:
        x1 = x - f(x)/df(x)
        if abs(x1 - x) < tol:  # Root is very close
            return x1, n
        else:
            x = x1 
            n += 1
        return None, n


## The secant method
The secant method uses secant lines to find the root. 

Find $a$ and $b$ such that,
$$ y = \frac{f(b)-f(a)}{b-a}(c-b)+f(b) = 0$$
the solution is:
$$ c = b-f(b)\frac{b-a}{f(b)-f(a)}$$

The secant method performs with fewer iterations compared to the bisection method, but with more than Newton's method.


In [11]:

def secant(f, a, b, tol=0.001, maxiter=100):
    """
    :param f: The function to solve
    :param a: Initial x-axis guess value
    :param b: Initial x-axis guess value, where b>a
    :param tol: The precision of the solution
    :param maxiter: Maximum number of iterations
    :return: The x-axis value of the root,
                number of iterations used
    """
    n= 1
    while n <= maxiter:
        c = b - f(b)*((b-a)/(f(b)-f(a)))
        if abs(c-b) < tol:
            return c, n
        a= b
        b= c
        n += 1
    return None, n

## SciPy implementations
Some root-finding functions that can be found in the `scipy.optimize` modules are `bisect`, `newton`, `brentq`, and `ridder`

In [13]:

"""
Documentation at
http://docs.scipy.org/doc/scipy/reference/optimize.html
"""
import scipy.optimize as optimize
y = lambda x: x**3 + 2.*x**2 - 5.
dy = lambda x: 3.*x**2 + 4.*x
# Call method: bisect(f, a, b[, args, xtol, rtol, maxiter, ...])
print("Bisection method: %s" % optimize.bisect(y, -5., 5., xtol=0.00001)) 

# Call method: newton(func, x0[, fprime, args, tol, ...])
print("Newton's method: %s" % optimize.newton(y, 5., fprime=dy)) 

# When fprime=None, then the secant method is used.
print("Secant method: %s" % optimize.newton(y, 5.)) 

# Call method: brentq(f, a, b[, args, xtol, rtol, maxiter, ...])
print("Brent's method: %s" % optimize.brentq(y, -5., 5.)) 

Bisection method: 1.241903305053711
Newton's method: 1.2418965630344798
Secant method: 1.2418965630344803
Brent's method: 1.241896563034559


## General nonlinear solvers
The scipy.optimize module also contains multidimensional general solvers:

`root(fun, x0[, args, method, jac, tol, ...])`: This finds a root of a vector function

`fsolve(func, x0[, args, fprime, ...])`: This finds the roots of a function